# Data Preparation

Die im Rahmen des Data Understanding gewonnenen Erkenntnisse über die Qualität der Daten werden nun gezielt eingesetzt, um eine Datenbereinigung und Vorverarbeitung für die Modellierung durchzuführen. Dadurch wird sichergestellt, dass die Daten qualitativ hochwertig und geeignet für den Einsatz in den Modellen sind. Dies ermöglicht eine robuste und aussagekräftige Analyse sowie eine zuverlässige Modellbildung.

#### Inhalt

- [Data Cleaning](#data_cleaning)
- [Preprocessing Pipeline](#preprocessing_pipeline)

---

## Data Cleaning <a name="data_cleaning"></a>

Es wurde festgestellt, dass der vorliegende Datensatz nicht vollständig den vorgegebenen Anforderungen entspricht und Mängel in der Datenqualität aufweist. Um diese Mängel zu beheben und den Datensatz zu verbessern, wird nun der Data Cleaning Prozess durchgeführt. Das übergeordnete Ziel des Data Cleaning besteht darin, den Datensatz so zu bereinigen, dass er für die nachfolgende Preprocessing Pipeline optimal vorbereitet ist. 

#### 🧩 Data Cleaning 

In [1]:
%%script false
from src.features.data_cleaning import *

# run data cleaning
pipeline = CleaningPipeline(path='../data/raw/twitter_tweets_raw.feather')
df = pipeline.run()

# save cleaned data set
df.to_feather('../data/intermediate/twitter_tweets_intermediate.feather')

2023-06-28 09:44:59,835 - INFO - initialize pipeline and load raw dataframe...
2023-06-28 09:45:01,727 - WARNING - starting data cleaning...
2023-06-28 09:45:01,727 - INFO - formating date...
2023-06-28 09:45:02,081 - WARNING - duplicate entries were found!
2023-06-28 09:45:02,081 - INFO - clean _no_duplicates...
2023-06-28 09:45:02,376 - WARNING - entries which are not in english were found!
2023-06-28 09:45:02,376 - INFO - clean _all_texts_in_english...
2023-06-28 09:45:02,679 - INFO - data cleaning completed successfully!


#### 0. Datensatz laden

In [1]:
import pandas as pd

df = pd.read_feather('../data/raw/twitter_tweets_raw.feather')

#### 1. Duplikate löschen 

In [2]:
df.drop_duplicates(subset=['rawContent'], inplace=True)

# check for success
if df['rawContent'].duplicated().any():
    print(f"{len(df[df['rawContent'].duplicated()])} duplicates found.")

#### 2. Nicht-englische Beiträge identifizieren & löschen 

In [3]:
from src.utils import logger

non_english_posts = df.query('lang != "en"')
logger.info(f'{len(non_english_posts)} non-English tweets found! drop...')
df.drop(index=non_english_posts.index, inplace=True)

# check for success
if not df['lang'].eq('en').all():
    print(df.query('lang != "en"'))

2023-06-30 11:53:01,368 - INFO - 1535 non-English tweets found! drop...


#### 3. Date aktualisieren

In [4]:
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

#### 4. Irrelevante Daten löschen

In [5]:
df.drop(columns=['lang', 'replyCount', 'retweetCount', 'likeCount'], inplace=True)

#### 5. Gesäuberten Datensatz speichern

In [9]:
df.set_index('url', inplace=True)
df.reset_index(inplace=True)
df.to_feather('../data/intermediate/twitter_tweets_intermediate.feather')

---

## Preprocessing Pipeline <a name="preprocessing_pipeline"></a>

Nachfolgend werden die enthaltenen Textdaten einer speziell entwickelten Preprocessing Pipeline zugeführt. Die Durchführung der Preprocessing Pipeline hat einen entscheidenden Einfluss auf die Qualität und das spätere Ergebnis der Modellierungen. Die Hauptaufgabe der Preprocessing Pipeline besteht darin, sicherzustellen, dass die Textdaten für die Modellierung geeignet und optimal vorbereitet sind. Die speziell für die Anforderungen dieses Projekts entworfene Pipeline kann visuell wie folgt dargestellt werden:

TODO: ABBILDUNG VON DER PREPROCESSING PIPELINE

#### 🧩 Preprocessing Pipeline ☕

In [13]:
%%script false
from src.features.preprocessing_pipeline import *
import pandas as pd

# run preprocessing pipeline
pipeline = DefaultPipeline(dataframe=pd.read_feather('../data/intermediate/twitter_tweets_intermediate.feather'))
df = pipeline.run()

# save preprocessed data set
df.to_feather('../data/processed/twitter_tweets_processed_2.feather')
df.to_csv('../data/processed/twitter_tweets_processed_2.csv', index=False)

2023-06-28 10:25:22,981 - INFO - initialize pipeline and download required nltk packages...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-06-28 10:25:23,086 - WARNING - starting default preprocessing pipeline
2023-06-28 10:25:23,086 - INFO - remove urls...
100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:15<00:00, 63413.64it/s]
2023-06-28 10:25:38,302 - INFO - remove twitter user mentions...
100%|█████████████████████████████████████████████████████████████████████| 960217/960217 [00:04<00:00, 230240.30it/s]
2023-06-28 10

#### 0. Packages & Datensatz laden 

In [1]:
import pandas as pd
import contractions
import nltk
import string
import emoji
import re

from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer

# download required nltk packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# enable progress bar
tqdm.pandas()

# load dataframe
df = pd.read_feather('../data/intermediate/twitter_tweets_intermediate.feather')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### 1.  URLs entfernen

In [2]:
def remove_urls(text):
    # define regex pattern for url detection
    url_pattern = re.compile(r'\b(?:https?://)?(?:[a-z]+\.[a-z]+\.[a-z]+|[a-z]+\.[a-z]+(?:/[^\s]*)?)\b')
    # remove url matches from the text
    text_without_urls = re.sub(url_pattern, '', text)
    return text_without_urls

df['preprocessed_text'] = df['rawContent'].progress_apply(remove_urls)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:14<00:00, 65563.58it/s]


#### 2.  Erwähnungen entfernen

In [3]:
def remove_mentions(text):
    # define regex pattern for user mentions
    mention_pattern = re.compile(r'@\w+')
    # remove user mentions from the text
    text_without_mentions = re.sub(mention_pattern, '', text)
    return text_without_mentions

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(remove_mentions)

100%|█████████████████████████████████████████████████████████████████████| 960217/960217 [00:03<00:00, 305004.41it/s]


#### 3. Kontraktionen auflösen

In [4]:
def fix_contractions(text):
    try:
        return contractions.fix(text)
    except IndexError: # error should not appear
        return text

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(fix_contractions)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:12<00:00, 78139.59it/s]


#### 4. Tokenization durchführen

In [5]:
# define tokenizer function
tokenizer = WordPunctTokenizer()

def tokenize_text(text):
    return tokenizer.tokenize(text)

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(tokenize_text)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:16<00:00, 57220.04it/s]


#### 5. Tokens in Kleinbuchstaben umwandeln

In [6]:
def lowercase(tokens):
    return [token.lower() for token in tokens]

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(lowercase)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:13<00:00, 70032.29it/s]


#### 6. Satzzeichen entfernen 

In [7]:
# adding more characters to the punctuation list
punct = string.punctuation + "’" + "``" +"`" + "''" +"'" + "•" + "“" + "”" + "…" + "�" + "‘" + "…" + "/…" + "-…" + "-#" + "’" + "..." + ".”" + "!!"

def remove_punct(tokens):
    return [token for token in tokens if token not in punct]

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(remove_punct)

100%|█████████████████████████████████████████████████████████████████████| 960217/960217 [00:07<00:00, 124969.19it/s]


#### 7.  Numerische Daten entfernen

In [8]:
def remove_numerics(tokens):
    return [token for token in tokens if not token.isdigit()]

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(remove_numerics)

100%|█████████████████████████████████████████████████████████████████████| 960217/960217 [00:05<00:00, 178545.31it/s]


#### 8.  Stopwörter entfernen

In [9]:
# define list of stopwords
stop_words = stopwords.words('english')

additional_stop_words = ['u']
stop_words.extend(additional_stop_words)

def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words and len(token) > 1]

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(remove_stopwords)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:46<00:00, 20701.79it/s]


#### 9.  Emojis entfernen

In [10]:
def remove_emoji(tokens):
    return [token for token in tokens if not any(char in emoji.EMOJI_DATA for char in token)]

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(remove_emoji)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [00:19<00:00, 48733.49it/s]


#### 10. Lemmatisierung durchführen

In [11]:
# initialization of the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['preprocessed_text'] = df['preprocessed_text'].progress_apply(lemmatize)

100%|██████████████████████████████████████████████████████████████████████| 960217/960217 [01:03<00:00, 15208.99it/s]


#### 11. Preprocessed Datensatz speichern

In [12]:
df.to_feather('../data/processed/twitter_tweets_processed.feather')
df.to_csv('../data/processed/twitter_tweets_processed.csv', index=False)

---